In [1]:
import pandas as pd
import numpy as np
import json

In [2]:
# Cargo el modelo y el catalogo de paises
data_neighbours = pd.read_csv('model/data_neighbours.csv', index_col=0)
data_matrix = pd.read_csv('model/data_matrix.csv', index_col=0)
catalog = pd.read_json('model/catalog.json', encoding ='utf-8')
catalog['code'] = catalog['code'].astype(str)
catalog['code'] = 't_' + catalog['code']

In [3]:
# Elijo una ciudad y me guardo el codigo
ciudad = ['Bangkok']
code = catalog[catalog['name'].isin(ciudad)]['code'].tolist()
code

['t_264']

In [4]:
# Recomiendo segun la ciudad que eligio
most_similar_to_likes = data_neighbours.loc[code]
similar_list = most_similar_to_likes.values.tolist()
similar_list = list(set([item for sublist in similar_list for item in sublist]))
neighbourhood = data_matrix[similar_list].loc[similar_list]

# A user vector containing only the neighbourhood items and the known user likes.
user_vector = pd.DataFrame(index=data_neighbours.index,columns=range(0,1))
for i in range(0,len(user_vector)):
    if user_vector.index[i] in code:
        user_vector.iloc[i,0] = 1.0
    else:
        user_vector.iloc[i,0] = 0.0

user_vector.iloc[:,0] = user_vector.iloc[:,0].astype(float)
user_vector = user_vector.loc[similar_list]

# Calculate the score.
score = neighbourhood.dot(user_vector.iloc[:,0]).div(neighbourhood.sum(axis=1))

# Drop the searched one.
score = score.drop(code)

print(code)
print(score.nlargest(20))

['t_264']
t_296    0.104798
t_205    0.065816
t_189    0.060334
t_355    0.058998
t_116    0.054511
t_113    0.053275
t_312    0.052456
t_222    0.052047
t_2      0.051884
dtype: float64


In [5]:
# Decodeo la respuesta
recomendacion = pd.DataFrame()
recomendacion['code'] = score.nlargest(20).index
recomendacion['score'] = np.array(score.nlargest(20))

recomendacion = recomendacion.merge(catalog, on='code')
recomendacion = recomendacion[['name','country','score']]
print(recomendacion)

json_recomendacion = recomendacion.to_json(orient='records')

name                             country     score
0        Phuket               {'name': 'Tailandia'}  0.104798
1         Dubái  {'name': 'Emiratos Árabes Unidos'}  0.065816
2    Chiang Mai               {'name': 'Tailandia'}  0.060334
3      Singapur                {'name': 'Singapur'}  0.058998
4         Tokio                   {'name': 'Japón'}  0.054511
5          Bali               {'name': 'Indonesia'}  0.053275
6        Cancún                  {'name': 'México'}  0.052456
7     Barcelona                  {'name': 'España'}  0.052047
8  Kuala Lumpur                 {'name': 'Malasia'}  0.051884
